# packages and imports

In [1]:
#imports
import pandas as pd
import numpy as np

from data_makers import *
import utils
from utils import *
import mean_model
from mean_model import meanModel
import ML_models
from ML_models import *

import tensorflow as tf
from alibi.explainers import IntegratedGradients

from sklearn.model_selection import train_test_split

from sklearn.metrics import mean_squared_error, r2_score

from sklearn.preprocessing import MinMaxScaler
from joblib import parallel_backend

from scipy.stats.stats import pearsonr

import shap

from Nikhil_code import data_preprocessing as dp_nb
from Nikhil_code import model_selection as ms_nb
from Nikhil_code import testing as t_nb
import Nikhil_code.Data_imports as di_nb
import Nikhil_code.pairs_train_test_split as tts_nb
import Nikhil_code.Learning_curve as lc_nb

2023-08-07 17:35:01.343135: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-07 17:35:01.501028: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-08-07 17:35:01.507459: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-08-07 17:35:01.507481: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if yo

## imaging

In [ ]:
# NN imaging imports

from skimage import data, io   # Import skimage library (data - Test images and example data.
#                          io - Reading, saving, and displaying images.)
from skimage.color import rgb2gray

import matplotlib.pyplot as plt                  # Import matplotlib.pyplot (Plotting framework in Python.)
%matplotlib inline
import os                                        # This module provides a portable way of using operating system dependent functionality.
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import warnings
# Suppress warnings
warnings.filterwarnings('ignore')
from IPython.display import display
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from tensorflow.keras import utils
from tensorflow.keras.models import Sequential,load_model
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, BatchNormalization
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
import seaborn as sns

# Feature selection (FS) and data creating for each drug

In [2]:
#input data
prot, rna, phospho_ls, one_hot_cls, one_hot_drugs, ic50_df1 = di_nb.read_input_data()
_all_cls = prot.index
_all_drugs = ic50_df1.columns
assert prot.shape[0] == rna.shape[0] == phospho_ls.shape[0]
assert phospho_ls.shape[0]  == one_hot_cls.shape[0]
prot.shape, rna.shape, phospho_ls.shape, one_hot_cls.shape, one_hot_drugs.shape

Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.


Number of missing prot values 0.386335609896865
num non overlapping prot and target cls: 10


Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.


num non overlapping cls: 930


((38, 8457), (38, 17417), (38, 22804), (38, 38), (345, 345))

In [3]:
## PHOSPHO
landmark_genes = pd.read_csv(
    f'data/Landmark_genes_LINCS.txt',sep='\t')
landmark_genes.index = landmark_genes['Symbol']

X_L1000 = landmark_X_maker(phospho_ls, landmark_genes['Symbol'])

#create prot data for all drugs
x_all_phos, x_drug, y_main_phos = dp_nb.create_all_drugs(
    X_L1000, one_hot_drugs, ic50_df1, _all_cls)

#fmt index to include drug cell line paris
cls_drugs_index = x_all_phos.index + '::' + x_drug.index 
x_all_phos.index = cls_drugs_index
y_main_phos.index = cls_drugs_index
x_drug.index = cls_drugs_index

x_all_phos = x_all_phos.astype(np.float32)
print(x_all_phos.shape)
X_main_phos = X_main_maker(x_all_phos, x_drug, short = False)
print(X_main_phos.shape)

(11583, 2056)
(11583, 2401)


In [ ]:
##PROTEOMIC
#use the same landmark genes for fs with prot data
landmark_genes = pd.read_csv(
    f'../data/Landmark_genes_LINCS.txt',sep='\t')
landmark_genes.index = landmark_genes['Symbol']

#find overlapping landmark genes and prot features
overlapping_landmarks, _ = dp_nb.keep_overlapping(
    pd.DataFrame(landmark_genes['Symbol']), prot.T)

overlapping_landmarks = overlapping_landmarks['Symbol'].values

#create prot data for all drugs
x_all_prot, x_drug, y_list = dp_nb.create_all_drugs(
    prot[overlapping_landmarks], one_hot_drugs, ic50_df1, _all_cls)

#fmt index to include drug cell line paris
cls_drugs_index = x_all_prot.index + '::' + x_drug.index 
x_all_prot.index = cls_drugs_index
y_list.index = cls_drugs_index
x_drug.index = cls_drugs_index

x_all_prot = x_all_prot.astype(np.float32)
x_all_prot.shape

In [4]:
#one hot data creation for all drugs
x_hot, x_drug_hot, y_main_onehot = dp_nb.create_all_drugs(
    one_hot_cls, one_hot_drugs, ic50_df1, _all_cls)

cls_drugs_index_hot = x_hot.index + '::' + x_drug_hot.index 

x_hot.index = cls_drugs_index_hot
x_hot.columns = np.arange(len(x_drug.columns), len(x_drug.columns) + len(x_hot.columns))
print(x_hot.shape)
X_main_onehot = X_main_maker(x_hot, x_drug, short = False)
print(X_main_onehot.shape)

(11583, 38)
(11583, 383)


# Early stopping and parameters

In [5]:
# set early stopping and learning rate scheduler
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=20, min_delta=0, restore_best_weights=True)
lr_scheduler = keras.callbacks.LearningRateScheduler(scheduler)
callbacks_list =  [es,lr_scheduler]

In [6]:
# set parameters and train
num_epochs = 200 # max epoch for training
learning_rate = 1e-2 # initial learning rate, decays via learning rate scheduler
momentum = 0.5
initializer_seed = 42

# Random Features

In [ ]:
#create dataframes that take a random set of features with the same length as landmark as a comparison
random_X = X_phos.sample(n=2056,axis='columns')

In [ ]:
## PHOSPHO
#create prot data for all drugs
x_all_phos, x_drug, y_main_phos = dp_nb.create_all_drugs(
    random_X, one_hot_drugs, ic50_df1, _all_cls)

#fmt index to include drug cell line paris
cls_drugs_index = x_all_phos.index + '::' + x_drug.index 
x_all_phos.index = cls_drugs_index
y_main_phos.index = cls_drugs_index
x_drug.index = cls_drugs_index

x_all_phos = x_all_phos.astype(np.float32)
print(x_all_phos.shape)
X_main_phos = X_main_maker(x_all_phos, x_drug, short = False)
print(X_main_phos.shape)

In [ ]:
#make the model a classifier
y_main_phos = classyFire(y_main_phos)

# Feature selection

In [ ]:
#read in the feature_importances_ feature selected data
feature_list = []
with open("feat_select_files/phospho/rfr_X_main/rfr_feat_select.txt", "r") as features:
    lines = features.readlines()
    for i in lines:
        i.replace(" ", "")
        feature_list.append(i.split(":")[0])

In [ ]:
#create a new X dataframe with the BEST selected features
X_features = phospho_ls.reindex(feature_list[:2000],axis="columns")

In [ ]:
#create a new X dataframe with the WORST selected features
X_features = X_phos.reindex(feature_list[21785:22785],axis="columns")

In [ ]:
## PHOSPHO
#create prot data for all drugs
x_all_phos, x_drug, y_main_phos = dp_nb.create_all_drugs(
    X_features, one_hot_drugs, ic50_df1, _all_cls)

#fmt index to include drug cell line paris
cls_drugs_index = x_all_phos.index + '::' + x_drug.index 
x_all_phos.index = cls_drugs_index
y_main_phos.index = cls_drugs_index
x_drug.index = cls_drugs_index

x_all_phos = x_all_phos.astype(np.float32)
print(x_all_phos.shape)
X_main_phos = X_main_maker(x_all_phos, x_drug, short = False)
print(X_main_phos.shape)

# train-test-split

## Early integration

In [7]:
#regular split
X_train, X_test, y_train, y_test = cell_line_split(X_main_phos, y_main_phos, test_size=0.2, random_state = 0)

In [ ]:
#short split
X_train_short, X_test_short, y_train_short, y_test_short = cell_line_split(X_main_phos[:1000], y_main_phos[:1000], test_size=0.2, random_state = 8)

## Late integration

In [8]:
#phospho
xo_train, xo_test = cell_line_split(x_all_phos, y=None, test_size=0.2, random_state = 0)
xd_train, xd_test = cell_line_split(x_drug, y=None, test_size=0.2, random_state = 0)
y_train, y_test = cell_line_split(y_main_phos, y=None, test_size=0.2, random_state = 0)

In [ ]:
#proteomic
xo_train, xo_test = cell_line_split(x_all_prot, y=None, test_size=0.2, random_state = 0)
xd_train, xd_test = cell_line_split(x_drug_prot, y=None, test_size=0.2, random_state = 0)
y_train, y_test = cell_line_split(y_main_prot, y=None, test_size=0.2, random_state = 0)

In [ ]:
#FOR TESTING NEW NETWORKS

x_all_phos = x_all_phos[:100].T[:100]
x_all_phos = x_all_phos.T

x_drug_phos = x_drug_phos[:100]

y_main_phos = y_main_phos[:100]

xo_train, xo_test = cell_line_split(x_all_phos, y=None, test_size=0.2, random_state = 0)
xd_train, xd_test = cell_line_split(x_drug_phos, y=None, test_size=0.2, random_state = 0)
y_train, y_test = cell_line_split(y_main_phos, y=None, test_size=0.2, random_state = 0)

# Convolutional Neural Network

model building and training

## Test models here

In [ ]:
## CNN using functional API
def build_early_integration_DNN_short(X_train, learning_rate, momentum, seed, mtype='regression'):

    # set layer weights initialiser
    initializer = keras.initializers.GlorotUniform(seed=seed)
    
    # drug-cell line data input
    x_input = layers.Input(shape=(X_train.shape[1]))

    # 5 fully connected layers with batch norm and dropout
    x = layers.Dense(16, kernel_initializer=initializer, activation='relu')(x_input)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.1)(x)
    x = layers.Dense(32, kernel_initializer=initializer, activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.1)(x)
    #x = layers.Dense(64, kernel_initializer=initializer, activation='relu')(x)
    #x = layers.BatchNormalization()(x)
    #x = layers.Dropout(0.1)(x)
    x = layers.Dense(32, kernel_initializer=initializer, activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.1)(x)
    x = layers.Dense(16, kernel_initializer=initializer, activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.1)(x)
    
    if mtype == 'classifier':
        output = layers.Dense(3, activation = 'softmax',activity_regularizer=keras.regularizers.l2())(x) # actiation layer is 3 neuron for each class

        model = keras.Model(x_input, output)

        model.compile( loss='sparse_categorical_crossentropy', optimizer=keras.optimizers.Adam(), metrics=['accuracy'])
    
    if mtype == 'regression':
        output = layers.Dense(1, kernel_initializer=initializer)(x) # actiation layer is 1 neuron for single regression prediction value
        
        model = keras.Model(x_input, output)
    
        model.compile(optimizer=keras.optimizers.RMSprop(learning_rate=learning_rate, 
                                                        momentum=momentum), 
                                                        loss='mse', metrics=['mae'])
    
    return model

In [ ]:
model = build_early_integration_DNN_short(X_train, learning_rate, momentum, initializer_seed, mtype='regression')


In [ ]:
model.summary()

## Early integration

In [ ]:
print(f'X_train shape: {X_train.shape}\ny_train shape: {y_train.shape}')

In [ ]:
%%time
## Code for model building and training 
 
model = build_early_integration_DNN(X_train, learning_rate, momentum, initializer_seed, mtype='regression')
history = model.fit(X_train, y_train,
                        validation_data=(X_test, y_test),
                        epochs=num_epochs, 
                        batch_size=None, 
                        verbose=0, 
                        callbacks=callbacks_list)

y_pred = model.predict(X_test)  

In [ ]:
print(f'length of y_pred: {len(y_pred)}\nlength of y_test:{len(y_test)}')

regression

In [ ]:
#regression metrics

r2 = r2_score(y_test, y_pred)
MSE = mean_squared_error(y_test, y_pred)
pearson = pearsonr(y_test, y_pred)

In [ ]:
print(f'r-squared: {r2}\nMSE: {MSE}\npearson: {pearson}')

classifier

In [ ]:
class_dict = {0:'High responsiveness', 1:'Intermediate responsiveness', 2:'Low responsiveness'}
class_list = [0, 1, 2]

In [ ]:
idx = 3026
print(f'For the screening of {y_test.index[idx]}\nThe prediction is:- high: {y_pred[idx][0]}, intermediate:- {y_pred[idx][1]}, low:- {y_pred[idx][2]}\nThe true value is: {y_test[idx]}')

In [ ]:
m = tf.keras.metrics.SparseCategoricalAccuracy()
m.update_state(y_test, y_pred)
m.result().numpy()

## Late integration

In [ ]:
print(f'xo_train shape: {xo_train.shape}\nxd_train shape: {xd_train.shape}\ny_train shape: {y_train.shape}')

In [ ]:
%%time
## Code for model building and training 
 
model = build_Deep_NN_tester(xo_train, xd_train, learning_rate, momentum, initializer_seed, complexity = 5)
history = model.fit([xo_train, xd_train], y_train,
                        validation_data=([xo_test, xd_test], y_test),
                        epochs=num_epochs, 
                        batch_size=None, 
                        verbose=0, 
                        callbacks=callbacks_list)

y_pred = model.predict([xo_test, xd_test])  


In [ ]:
print(f'length of y_pred: {len(y_pred)}\nlength of y_test:{len(y_test)}')

regressor

In [ ]:
r2 = r2_score(y_test, y_pred)
MSE = mean_squared_error(y_test, y_pred)
pearson = pearsonr(y_test, y_pred)

In [ ]:
print(f'r-squared: {r2}\nMSE: {MSE}\npearson: {pearson}')

classifier

In [ ]:
class_dict = {0:'High responsiveness', 1:'Intermediate responsiveness', 2:'Low responsiveness'}

In [ ]:
print(f'For the screening of {y_test.index[0]}\nThe prediction is:- high: {y_pred[0][0]}, intermediate:- {y_pred[0][1]}, low:- {y_pred[0][2]}\nThe true value is: {y_test[0]}')

In [ ]:
m = tf.keras.metrics.SparseCategoricalAccuracy()
m.update_state(y_test, y_pred)
m.result().numpy()

## Visualise networks

In [ ]:
#LATE INTEGRATION

learning_rate = 1e-2 # initial learning rate, decays via learning rate scheduler
momentum = 0.5
initializer_seed = 0
 
late_model = build_CNN(xo_train, xd_train, learning_rate, momentum, initializer_seed)

In [ ]:
#EARLY INTEGRATION

learning_rate = 1e-2 # initial learning rate, decays via learning rate scheduler
momentum = 0.5
initializer_seed = 42
 
early_model = build_early_integration_CNN(X_train, learning_rate, momentum, initializer_seed)

In [ ]:
late_model.summary()

In [ ]:
early_model.summary()

In [ ]:
from ann_visualizer.visualize import ann_viz

ann_viz(early_model, view=True, filename='construct_model', title='CNN — Model 1 — Simple Architecture')

In [ ]:
import visualkeras
from PIL import ImageFont

visualkeras.graph_view(early_model)

In [ ]:
%load_ext tensorboard
from datetime import datetime
from tensorflow import keras

# Define the Keras TensorBoard callback.
logdir="logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)

In [ ]:
# set early stopping and learning rate scheduler
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=20, min_delta=0, restore_best_weights=True)
lr_scheduler = tf.keras.callbacks.LearningRateScheduler(scheduler)

history = early_model.fit(X_train_short, y_train_short,
                        validation_data=(X_test_short, y_test_short),
                        epochs=200, 
                        batch_size=None, 
                        verbose=0, 
                        callbacks=[tensorboard_callback, es, lr_scheduler])

In [ ]:
logdir

## phospho

Late integration

In [ ]:
print(f'X_train shape: {xo_train.shape}\ny_train shape: {y_train.shape}')

In [ ]:
x_all_phos[:1]

In [ ]:
y_main_phos[:1]

In [ ]:
rs = [0, 8, 23, 42, 69, 88]
mm_r2 = []
mixed_set_r2 = []
model_r2 = []
model_MSE = []
model_pearson = []

for seed in rs:
    
    print(f'Seed: {seed}')
    
    print(f'Mean Model')
    #MEAN MODEL
    #tts
    X_train, X_test, y_train, y_test = cell_line_split(X_main_phos, y=y_main_phos, test_size=0.2, random_state = seed)
    #run mean model
    dl = dlMaker(y_train, noRepeats=True)
    mm = meanModel(y_train, dl)
    prediction = mm.predict(y_test.index)
    mean_r2 = r2_score(y_test, list(prediction.values()))
    mm_r2.append(mean_r2)
    
    
    
    
    print(f'Mixed-Set Model')
    #MIXED SET
    # one hot the dataframes
    onehotX = one_hot_maker(X_L1000, X_df=True)
    
    #produce X-main and y_main
    cl = clMaker(onehotX, y_phos)
    x_all_oneHot, x_drug_oneHot, y_main_oneHot = create_all_drugs(x=onehotX, xd=hotdrugsDF_phos, y=y_phos, cells=cl)
    
    #phospho
    xo_train, xo_test = cell_line_split(x_all_oneHot, y=None, test_size=0.2, random_state = seed)
    xd_train, xd_test = cell_line_split(x_drug_oneHot, y=None, test_size=0.2, random_state = seed)
    y_train, y_test = cell_line_split(y_main_oneHot, y=None, test_size=0.2, random_state = seed)
    
    ## Code for model building and training

    model = build_Deep_NN(xo_train, xd_train, learning_rate, momentum, initializer_seed, complexity = 1)
    history = model.fit([xo_train, xd_train], y_train,
                        validation_data=([xo_test, xd_test], y_test),
                        epochs=num_epochs, 
                        batch_size=None, 
                        verbose=0, 
                        callbacks=callbacks_list)

    y_pred = model.predict([xo_test, xd_test])  
    r2 = r2_score(y_test, y_pred)
        
    mixed_set_r2.append(r2)

    
    
    
    print(f'Model: CNN')
    
    #phospho
    xo_train, xo_test = cell_line_split(x_all_phos, y=None, test_size=0.2, random_state = seed)
    xd_train, xd_test = cell_line_split(x_drug_phos, y=None, test_size=0.2, random_state = seed)
    y_train, y_test = cell_line_split(y_main_phos, y=None, test_size=0.2, random_state = seed)
    

    ## Code for model building and training

    model = build_Deep_NN(xo_train, xd_train, learning_rate, momentum, initializer_seed, complexity = 1)
    history = model.fit([xo_train, xd_train], y_train,
                        validation_data=([xo_test, xd_test], y_test),
                        epochs=num_epochs, 
                        batch_size=None, 
                        verbose=0, 
                        callbacks=callbacks_list)

    y_pred = model.predict([xo_test, xd_test])  
    r2 = r2_score(y_test, y_pred)
    MSE = mean_squared_error(y_test, y_pred)
    pearson = pearsonr(y_test, y_pred)
    model_r2.append(r2)
    model_MSE.append(MSE)
    model_pearson.append(pearson)


In [ ]:
table_make('CNN_metrics',
           [rs, mm_r2, mixed_set_r2, model_r2, model_MSE, model_pearson],
           ['rs', 'mm_r2', 'mixed_set_r2', 'model_r2', 'model_MSE', 'model_pearson'], 
           file = 'plots/phospho/NormalisedX_LateDNN(1)_metrics_L1000.csv')
           

Early integration

In [ ]:
print(f'X_train shape: {X_train.shape}\ny_train shape: {y_train.shape}')

In [ ]:
rs = [0, 8, 23, 42, 69, 88]
mm_r2 = []
mixed_set_r2 = []
model_r2 = []
model_MSE = []
model_pearson = []

for seed in rs:
    
    print(f'Seed: {seed}')
    
    print(f'Mean Model')
    #MEAN MODEL
    #tts
    X_train, X_test, y_train, y_test = cell_line_split(X_main_phos, y=y_main_phos, test_size=0.2, random_state = seed)
    #run mean model
    dl = dlMaker(y_train, noRepeats=True)
    mm = meanModel(y_train, dl)
    prediction = mm.predict(y_test.index)
    mean_r2 = r2_score(y_test, list(prediction.values()))
    mm_r2.append(mean_r2)
    
    print(f'Mixed-Set Model')
    #MIXED SET
    # one hot the dataframes
    onehotX = one_hot_maker(X_L1000, X_df=True)
    
    #produce X-main and y_main
    cl = clMaker(onehotX, y_phos)
    x_all_oneHot, x_drug_oneHot, y_main_oneHot = create_all_drugs(x=onehotX, xd=hotdrugsDF_phos, y=y_phos, cells=cl)
    X_main_oneHot = X_main_maker(x_all_oneHot, x_drug_oneHot, short = False)
    
    #regular split
    X_train, X_test, y_train, y_test = cell_line_split(X_main_oneHot, y_main_oneHot, test_size=0.2, random_state = 0)

    ## Code for model building and training
    
    model = DeepIC50(X_train, learning_rate, momentum, initializer_seed, mtype='regression')
    history = model.fit(X_train, y_train,
                        validation_data=(X_test, y_test),
                        epochs=num_epochs, 
                        batch_size=None, 
                        verbose=0, 
                        callbacks=callbacks_list)

    y_pred = model.predict(X_test)  
    
    r2 = r2_score(y_test, y_pred)
    mixed_set_r2.append(r2)

    
    
    
    print(f'Model: CNN')
    
    #regular split
    X_train, X_test, y_train, y_test = cell_line_split(X_main_phos, y_main_phos, test_size=0.2, random_state = 0)

    ## Code for model building and training

    model = DeepIC50(X_train, learning_rate, momentum, initializer_seed, mtype='regression')
    history = model.fit(X_train, y_train,
                        validation_data=(X_test, y_test),
                        epochs=num_epochs, 
                        batch_size=None, 
                        verbose=0, 
                        callbacks=callbacks_list)

    y_pred = model.predict(X_test)  
    r2 = r2_score(y_test, y_pred)
    MSE = mean_squared_error(y_test, y_pred)
    pearson = pearsonr(y_test, y_pred)
    model_r2.append(r2)
    model_MSE.append(MSE)
    model_pearson.append(pearson)


In [ ]:
table_make('CNN_metrics',
           [rs, mm_r2, mixed_set_r2, model_r2, model_MSE, model_pearson],
           ['rs', 'mm_r2', 'mixed_set_r2', 'model_r2', 'model_MSE', 'model_pearson'], 
           file = 'plots/phospho/NormalisedX_DeepIC50_metrics_L1000.csv')
           

## proteomic

In [ ]:
rs = [0, 8, 23, 42, 69, 88]
mm_r2 = []
mixed_set_r2 = []
model_r2 = []
model_MSE = []
model_pearson = []

for seed in rs:
    
    print(f'Seed: {seed}')
    
    print(f'Mean Model')
    #MEAN MODEL
    #tts
    X_train, X_test, y_train, y_test = cell_line_split(X_main_prot, y=y_main_prot, test_size=0.2, random_state = seed)
    #run mean model
    dl = dlMaker(y_train, noRepeats=True)
    mm = meanModel(y_train, dl)
    prediction = mm.predict(y_test.index)
    mean_r2 = r2_score(y_test, list(prediction.values()))
    mm_r2.append(mean_r2)
    
    
    
    
    print(f'Mixed-Set Model')
    #MIXED SET
    # one hot the dataframes
    onehotX = one_hot_maker(X_prot, X_df=True)
    
    #produce X-main and y_main
    cl = clMaker(onehotX, y_prot)
    x_all_oneHot, x_drug_oneHot, y_main_oneHot = create_all_drugs(x=onehotX, xd=hotdrugsDF_prot, y=y_prot, cells=cl)
    
    #phospho
    xo_train, xo_test = cell_line_split(x_all_oneHot, y=None, test_size=0.2, random_state = seed)
    xd_train, xd_test = cell_line_split(x_drug_oneHot, y=None, test_size=0.2, random_state = seed)
    y_train, y_test = cell_line_split(y_main_oneHot, y=None, test_size=0.2, random_state = seed)

    ## Code for model building and training

    # set early stopping and learning rate scheduler
    es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=20, min_delta=0, restore_best_weights=True)
    lr_scheduler = tf.keras.callbacks.LearningRateScheduler(scheduler)
    callbacks_list =  [es,lr_scheduler]

    # set parameters and train
    num_epochs = 200 # max epoch for training
    learning_rate = 1e-2 # initial learning rate, decays via learning rate scheduler
    momentum = 0.5
    initializer_seed = 42

    model = build_Deep_NN(xo_train, xd_train, learning_rate, momentum, initializer_seed)
    history = model.fit([xo_train, xd_train], y_train,
                        validation_data=([xo_test, xd_test], y_test),
                        epochs=num_epochs, 
                        batch_size=None, 
                        verbose=0, 
                        callbacks=callbacks_list)

    y_pred = model.predict([xo_test, xd_test])  
    r2 = r2_score(y_test, y_pred)
        
    mixed_set_r2.append(r2)

    
    
    
    print(f'Model: CNN')
    
    #phospho
    xo_train, xo_test = cell_line_split(x_all_prot, y=None, test_size=0.2, random_state = seed)
    xd_train, xd_test = cell_line_split(x_drug_prot, y=None, test_size=0.2, random_state = seed)
    y_train, y_test = cell_line_split(y_main_prot, y=None, test_size=0.2, random_state = seed)
    

    ## Code for model building and training

    # set early stopping and learning rate scheduler
    es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=20, min_delta=0, restore_best_weights=True)
    lr_scheduler = tf.keras.callbacks.LearningRateScheduler(scheduler)
    callbacks_list =  [es,lr_scheduler]

    # set parameters and train
    num_epochs = 200 # max epoch for training
    learning_rate = 1e-2 # initial learning rate, decays via learning rate scheduler
    momentum = 0.5
    initializer_seed = 42

    model = build_Deep_NN(xo_train, xd_train, learning_rate, momentum, initializer_seed)
    history = model.fit([xo_train, xd_train], y_train,
                        validation_data=([xo_test, xd_test], y_test),
                        epochs=num_epochs, 
                        batch_size=None, 
                        verbose=0, 
                        callbacks=callbacks_list)

    y_pred = model.predict([xo_test, xd_test])  
    r2 = r2_score(y_test, y_pred)
    MSE = mean_squared_error(y_test, y_pred)
    pearson = pearsonr(y_test, y_pred)
    model_r2.append(r2)
    model_MSE.append(MSE)
    model_pearson.append(pearson)


In [ ]:
table_make('Deep_NN_metrics_prot',
           [rs, mm_r2, mixed_set_r2, model_r2, model_MSE, model_pearson],
           ['rs', 'mm_r2', 'mixed_set_r2', 'model_r2', 'model_MSE', 'model_pearson'], 
           file = 'plots/proteomic/Deep_NN_metrics.csv')
           

## mixed set model

In [ ]:
## Code for model building and training

# set early stopping and learning rate scheduler
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=20, min_delta=0, restore_best_weights=True)
lr_scheduler = tf.keras.callbacks.LearningRateScheduler(scheduler)
callbacks_list =  [es,lr_scheduler]

# set parameters and train
num_epochs = 200 # max epoch for training
learning_rate = 1e-2 # initial learning rate, decays via learning rate scheduler
momentum = 0.5
initializer_seed = 42

model = build_Deep_NN(xo_train, xd_train, learning_rate, momentum, initializer_seed)
history = model.fit([xo_train, xd_train], y_train,
                    validation_data=([xo_test, xd_test], y_test),
                    epochs=num_epochs, 
                    batch_size=None, 
                    verbose=0, 
                    callbacks=callbacks_list)

y_pred = model.predict([xo_test, xd_test])  
r2 = r2_score(y_test, y_pred)


# SHAP for NN

In [11]:
print(f'X_train shape: {xo_train.shape}')
print(f'X_train shape: {xd_train.shape}')
print(f'y_train shape: {y_train.shape}')

X_train shape: (9009, 2056)
X_train shape: (9009, 345)
y_train shape: (9009,)


In [12]:
%%time
## fit the model

model5 = build_CNN(xo_train, xd_train, learning_rate, momentum, initializer_seed, mtype = 'regression')
history = model5.fit([xo_train, xd_train], y_train,
                    validation_data=([xo_test, xd_test], y_test),
                    epochs=num_epochs, 
                    batch_size=None, 
                    verbose=0, 
                    callbacks=callbacks_list)

2023-08-07 17:38:13.754196: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Restoring model weights from the end of the best epoch: 28.
Epoch 48: early stopping
CPU times: user 12min 35s, sys: 26.1 s, total: 13min 1s
Wall time: 3min 30s


In [34]:
# DeepExplainer to explain predictions of the model
explainer = shap.GradientExplainer(model5, [xo_train, xd_train])

In [35]:
#specify the cell line and drugs that are tested
idx = 0

cell_line, drug = list(xo_test.index)[idx].split('::')

print(f'cell line: {cell_line}\ndrug: {drug}\nIC50: {y_test[f"{cell_line}::{drug}"]}')

cell line: KYSE-450
drug: (5Z)-7-Oxozeaenol
IC50: 1.708192


In [ ]:
%%time
# make a shap value array for the prediction
shaps = explainer.shap_values([np.array([xo_test.iloc[0]]), np.array([xd_test.iloc[0]])])


# Permutation testing

In [ ]:
from alibi.explainers import PermutationImportance, plot_permutation_importance

In [ ]:
print(f'X_train shape: {X_train.shape}\ny_train shape: {y_train.shape}')

In [ ]:
%%time
## fit the model

#build model
model6 = build_early_integration_DNN_short(X_train, learning_rate, momentum, initializer_seed, mtype='regression')
history = model6.fit(X_train, y_train,
                        validation_data=(X_test, y_test),
                        epochs=num_epochs, 
                        batch_size=None, 
                        verbose=0, 
                        callbacks=callbacks_list)

In [ ]:
def predict_fn(X_train):
    return model6.predict(X_train)

In [ ]:
explainer = PermutationImportance(predictor=predict_fn,
                                  score_fns=['r2'],
                                  feature_names=list(X_train.columns),
                                  verbose=True)

In [ ]:
#specify the cell line and drugs that are tested
idx = 0

cell_line, drug = list(xo_test.index)[idx].split('::')

print(f'cell line: {cell_line}\ndrug: {drug}\nIC50: {y_test[f"{cell_line}::{drug}"]}')

In [ ]:
%%time
exp = explainer.explain(X=X_test.values, y=y_test.values)

In [ ]:
feat_importances = []
for i, feat in enumerate(exp.feature_importance[0][:100]):
    feat_importances.append(f'{exp.feature_names[:100][i]}: {list(feat.values())[0]}')
feat_importances

# Integrated Gradients

In [37]:
print(f'X_train shape: {X_train.shape}\ny_train shape: {y_train.shape}')

X_train shape: (9009, 2401)
y_train shape: (9009,)


Early integration DNN

In [38]:
%%time

#build model
model1 = build_early_integration_DNN(X_train, learning_rate, momentum, initializer_seed, mtype='regression')

#fit model
history = model1.fit(X_train, y_train,
                        validation_data=(X_test, y_test),
                        epochs=num_epochs, 
                        batch_size=None, 
                        verbose=0, 
                        callbacks=callbacks_list)
#initialise IG explainer
ig1  = IntegratedGradients(model1,
                          layer=None,
                          target_fn=None,
                          method="gausslegendre",
                          n_steps=50,
                          internal_batch_size=100)

Restoring model weights from the end of the best epoch: 25.
Epoch 45: early stopping
CPU times: user 12min 12s, sys: 15.7 s, total: 12min 28s
Wall time: 3min 15s


Early integration DeepIC50

In [39]:
%%time

#build model
model2 = DeepIC50(X_train, learning_rate, momentum, initializer_seed, mtype='regression')

#fit model
history = model2.fit(X_train, y_train,
                        validation_data=(X_test, y_test),
                        epochs=num_epochs, 
                        batch_size=None, 
                        verbose=0, 
                        callbacks=callbacks_list)

#Initialise IG explainer
ig2  = IntegratedGradients(model2,
                          layer=None,
                          target_fn=None,
                          method="gausslegendre",
                          n_steps=50,
                          internal_batch_size=100)

Restoring model weights from the end of the best epoch: 57.
Epoch 77: early stopping
CPU times: user 1h 20min 48s, sys: 1min 39s, total: 1h 22min 28s
Wall time: 20min 31s


Late integration DNN

In [40]:
%%time
 
#build model
model3 = build_Deep_NN(xo_train, xd_train, learning_rate, momentum, initializer_seed, complexity = 4)

#fit model
history = model3.fit([xo_train, xd_train], y_train,
                        validation_data=([xo_test, xd_test], y_test),
                        epochs=num_epochs, 
                        batch_size=None, 
                        verbose=0, 
                        callbacks=callbacks_list)

#Initialise IG explainer
ig3  = IntegratedGradients(model3,
                          layer=None,
                          target_fn=None,
                          method="gausslegendre",
                          n_steps=50,
                          internal_batch_size=100)

Restoring model weights from the end of the best epoch: 15.
Epoch 35: early stopping
CPU times: user 1min 27s, sys: 6.03 s, total: 1min 33s
Wall time: 40 s


Late integration CNN

In [41]:
%%time
#build model
model4 = build_CNN(xo_train, xd_train, learning_rate, momentum, initializer_seed, mtype = 'regression')

#fit model
history = model4.fit([xo_train, xd_train], y_train,
                        validation_data=([xo_test, xd_test], y_test),
                        epochs=num_epochs, 
                        batch_size=None, 
                        verbose=0, 
                        callbacks=callbacks_list)

#Initialise IG explainer
ig4  = IntegratedGradients(model4,
                          layer=None,
                          target_fn=None,
                          method="gausslegendre",
                          n_steps=50,
                          internal_batch_size=100)

Restoring model weights from the end of the best epoch: 24.
Epoch 44: early stopping
CPU times: user 11min 6s, sys: 21.8 s, total: 11min 28s
Wall time: 3min 4s


In [42]:
idx = 2
num_feats = 100

#specify the cell line and drugs that are tested

cell_line, drug = list(xo_test.index)[idx].split('::')

print(f'cell line: {cell_line}\ndrug: {drug}')

cell line: ML-2
drug: (5Z)-7-Oxozeaenol


Early integration DNN

In [43]:
#IC50 prediction
y_pred = model1.predict(X_test[idx:(idx+1)])  
print('------------------------------------------------')
print(f'True IC50 value: {y_train[idx:(idx+1)].values[0]}\nPredicted IC50 value: {y_pred[0][0]}')

#explainer instance
explanation = ig1.explain(X_train[idx:(idx+1)].values,
                         baselines=None,
                         target=None)

#print the top features for this explanation 
attributions = explanation.attributions
#this function outputs the top x number of features and their scores for a model
print('------------------------------------------------')
NN_final_names, NN_final_scores = rfrFeatures(attributions[0][0], X_main = X_train, topX = num_feats, N = num_feats)
for i in range(10):
    print(f'{NN_final_names[i]}:{NN_final_scores[i]}')
print('------------------------------------------------')

1/1 [==============================] - 0s 157ms/step


It looks like you are passing a model with a scalar output and target is set to `None`.If your model is a regression model this will produce correct attributions. If your model is a classification model, targets for each datapoint must be defined. Not defining the target may lead to incorrect values for the attributions.Targets can be either the true classes or the classes predicted by the model.


------------------------------------------------
True IC50 value: 0.948509
Predicted IC50 value: 1.2860968112945557
------------------------------------------------
CDK1(S233);:0.01420361208843644
MAP7(S365);:0.007983334899808334
SPP1(S215);:0.00792561300044159
MSH6(S43);:0.006931786660178099
NOSIP(S107);:0.006218817333435036
RAP1GAP(T486);:0.005021102113867307
RAP1GAP(S484);:0.004924069337543032
IGF2R(S2400);IGF2R(S2401);:0.004649320641185236
RRP8(S51);RRP8(S53);:0.0037547182103572527
KDM3A(S463);:0.0031338973673540236
------------------------------------------------


Early integration DeepIC50

In [44]:
#IC50 prediction
y_pred = model2.predict(X_test[idx:(idx+1)])  
print('------------------------------------------------')
print(f'True IC50 value: {y_train[idx:(idx+1)].values[0]}\nPredicted IC50 value: {y_pred[0][0]}')

#explainer instance
explanation = ig2.explain(X_train[idx:(idx+1)].values,
                         baselines=None,
                         target=None)

#print the top features for this explanation 
attributions = explanation.attributions
#this function outputs the top x number of features and their scores for a model
print('------------------------------------------------')
NN_final_names, NN_final_scores = rfrFeatures(attributions[0][0], X_main = X_train, topX = num_feats, N = num_feats)
for i in range(10):
    print(f'{NN_final_names[i]}:{NN_final_scores[i]}')
print('------------------------------------------------')

1/1 [==============================] - 0s 281ms/step


It looks like you are passing a model with a scalar output and target is set to `None`.If your model is a regression model this will produce correct attributions. If your model is a classification model, targets for each datapoint must be defined. Not defining the target may lead to incorrect values for the attributions.Targets can be either the true classes or the classes predicted by the model.


------------------------------------------------
True IC50 value: 0.948509
Predicted IC50 value: 2.1359477043151855
------------------------------------------------
CCDC86(S37);:50.08771652675226
IGF2R(S2400);IGF2R(S2401);:47.53724513202866
IGF2R(T2415);:44.248619555893484
DDX42(S751);DDX42(S754);:27.72710106155702
GLI2(S236);:23.17818429152722
IGF2R(S2409);:20.230175341507017
SPP1(S219);:19.339562999115902
EPN2(S195);:14.837722069198291
CCDC86(S58);:14.098201273561386
EML3(S177);:13.590920264559813
------------------------------------------------


Late integration DNN

In [45]:
#IC50 prediction
y_pred = model3.predict([xo_test[idx:(idx+1)], xd_test[idx:(idx+1)]])  
print('------------------------------------------------')
print(f'True IC50 value: {y_train[idx:(idx+1)].values[0]}\nPredicted IC50 value: {y_pred[0][0]}')

#explainer instance
explanation = ig3.explain([xo_train[idx:(idx+1)].values, np.array(xd_train[idx:(idx+1)])],
                         baselines=None,
                         target=None)

#print the top features for this explanation
attributions = explanation.attributions
#this function outputs the top x number of features and their scores for a model
print('------------------------------------------------')
NN_final_names, NN_final_scores = rfrFeatures(attributions[0][0], X_main = X_train, topX = num_feats, N = num_feats)
for i in range(10):
    print(f'{NN_final_names[i]}:{NN_final_scores[i]}')
print('------------------------------------------------')

1/1 [==============================] - 0s 108ms/step


It looks like you are passing a model with a scalar output and target is set to `None`.If your model is a regression model this will produce correct attributions. If your model is a classification model, targets for each datapoint must be defined. Not defining the target may lead to incorrect values for the attributions.Targets can be either the true classes or the classes predicted by the model.


------------------------------------------------
True IC50 value: 0.948509
Predicted IC50 value: -0.5204095244407654
------------------------------------------------
CDK1(S233);:0.016885131329390082
MAP7(S365);:0.010485951762391822
NOSIP(S107);:0.010390026537665449
SPP1(S215);:0.008735702721325291
MYCBP2(S2774);:0.00822409793527249
RAP1GAP(T486);:0.006853564907783864
RAP1GAP(S484);:0.0060732027869332805
MSH6(S43);:0.0053999583465289165
RRP8(S51);RRP8(S53);:0.005390667881927584
KAT6B(S1597);KAT6B(S1598);:0.004900299172853961
------------------------------------------------


Late integration CNN

In [46]:
#IC50 prediction
y_pred = model4.predict([xo_test[idx:(idx+1)], xd_test[idx:(idx+1)]]) 
print('------------------------------------------------')
print(f'True IC50 value: {y_train[idx:(idx+1)].values[0]}\nPredicted IC50 value: {y_pred[0][0]}')

#explainer instance
explanation = ig4.explain([xo_train[idx:(idx+1)].values, np.array(xd_train[idx:(idx+1)])],
                         baselines=None,
                         target=None)
#print the top features for this explanation
attributions = explanation.attributions
#this function outputs the top x number of features and their scores for a model
print('------------------------------------------------')
NN_final_names, NN_final_scores = rfrFeatures(attributions[0][0], X_main = X_train, topX = num_feats, N = num_feats)
for i in range(10):
    print(f'{NN_final_names[i]}:{NN_final_scores[i]}')
print('------------------------------------------------')

1/1 [==============================] - 0s 166ms/step


It looks like you are passing a model with a scalar output and target is set to `None`.If your model is a regression model this will produce correct attributions. If your model is a classification model, targets for each datapoint must be defined. Not defining the target may lead to incorrect values for the attributions.Targets can be either the true classes or the classes predicted by the model.


------------------------------------------------
True IC50 value: 0.948509
Predicted IC50 value: -0.7935070991516113
------------------------------------------------
DDX10(S793);:7.974487250216398
JUN(S63);:7.917300081603442
KAT6B(S1597);KAT6B(S1598);:5.064132690122426
KAT6B(S1581);:3.9622710298443082
CDK1(S233);:3.486010492916559
IKZF1(S442);:3.157174011427876
MACF1(S5718);:3.080785043651632
HOXA10(S282);:1.630120018788577
CCDC86(S37);:1.512939385380748
CRTAP(S250);:1.38290933649561
------------------------------------------------


# Performance plots

In [ ]:
metrics1 = pd.read_csv('plots/phospho/NormalisedX_CNN_metrics_randomFeats.csv')
metrics2 = pd.read_csv('plots/phospho/NormalisedX_CNN_metrics_L1000.csv')
#xgb_metrics = pd.read_csv('plots/phospho/xgb_metrics.csv')

fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Plot 1
axes[0].plot(metrics1['rs'].astype(str), metrics1['mm_r2'], linestyle='-', marker='.', color='orange', linewidth=1)
axes[0].plot(metrics1['rs'].astype(str), metrics1['mixed_set_r2'], linestyle='-', marker='.', color='yellow', linewidth=1)
axes[0].plot(metrics1['rs'].astype(str), metrics1['model_r2'], linestyle='-', marker='.', color='blue', linewidth=1)
axes[0].set_title('Random Features')

# Plot 2
axes[1].plot(metrics2['rs'].astype(str), metrics2['mm_r2'], linestyle='-', marker='.', color='orange', linewidth=1)
axes[1].plot(metrics2['rs'].astype(str), metrics2['mixed_set_r2'], linestyle='-', marker='.', color='yellow', linewidth=1)
axes[1].plot(metrics2['rs'].astype(str), metrics2['model_r2'], linestyle='-', marker='.', color='blue', linewidth=1)
axes[1].set_title('L1000')

# Plot 3
#axes[2].plot(shap_df_features['rs'].astype(str), shap_df_features['mm_r2'], linestyle='-', marker='.', color='orange', linewidth=1)
#axes[2].plot(shap_df_features['rs'].astype(str), shap_df_features['model_r2'], linestyle='-', marker='.', color='blue', linewidth=1)
#axes[2].set_title('Features Filtered')

# Adjust spacing between subplots
plt.tight_layout()

# Display the plots
plt.show()

In [ ]:
metrics1 = pd.read_csv('plots/phospho/NormalisedX_CNN_metrics_randomFeats.csv')
metrics2 = pd.read_csv('plots/phospho/NormalisedX_CNN_metrics_L1000.csv')
#xgb_metrics = pd.read_csv('plots/phospho/xgb_metrics.csv')

fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Plot 1
axes[0].plot(metrics1['rs'].astype(str), metrics1['model_r2'], linestyle='-', marker='.', color='yellow', linewidth=1)
axes[0].plot(metrics2['rs'].astype(str), metrics2['model_r2'], linestyle='-', marker='.', color='blue', linewidth=1)
axes[0].set_title('Random Features')

# Adjust spacing between subplots
plt.tight_layout()

# Display the plots
plt.show()